In [ ]:
from influxdb_client import InfluxDBClient
import pandas as pd
import os
import warnings
import matplotlib.pyplot as plt
warnings.filterwarnings("ignore")

## Connessione al database InfluxDB
INFLUXDB_URL = "http://localhost:8086"
TOKEN = os.getenv("token")  
ORG = "IotAlarmSystem"
BUCKET = "Prova"
client = InfluxDBClient(url=INFLUXDB_URL, token=TOKEN, org=ORG)
query_api = client.query_api()

## Query cambiare i giorni per prendere più o meno dati 
query = f"""
from(bucket: "{BUCKET}")
|> range(start: -90d)
|> filter(fn: (r) => r._measurement == "pressure")
|> filter(fn: (r) => r._field == "value")
|> keep(columns: ["_time", "_value"])
"""

result = query_api.query(query)
data = []



for table in result:
    for record in table.records:
        data.append({"time": record["_time"], "value": record["_value"]})


#A seconda della fonte dati cambiare l'assegnazioene di df 

# df = pd.DataFrame(data)
df = pd.read_csv("simulated_data_random.csv")

In [ ]:
def sleep_per_night(df):
    """
    Calcolo tempo totale di sonno per ogni notte, considerando la notte dalle 21:00 del giorno X 
    alle 10:00 del giorno X+1 e ignorando i dati al di fuori di questo intervallo.
    """

    df["time"] = pd.to_datetime(df["time"])
    df["night"] = df["time"].dt.date
    df["hour"] = df["time"].dt.hour


    df.loc[df["hour"] < 10, "night"] = df["night"] - pd.Timedelta(days=1)
    df = df[(df["hour"] >= 21) | (df["hour"] < 10)]

    sleep_data = []


    for night, night_df in df.groupby("night"):
        sleep_seconds = 0
        i = 0
        sleep_sessions = []

        while i < len(night_df):
            while i < len(night_df) and night_df.iloc[i]["value"] != 4095:
                i += 1
            if i >= len(night_df):
                break

            start_time = night_df.iloc[i]["time"]

            # Avanzare fino a trovare il primo valore diverso da 4095
            while i < len(night_df) and night_df.iloc[i]["value"] == 4095:
                i += 1
            if i >= len(night_df):
                break

            # Salvare la fine del sonno
            end_time = night_df.iloc[i]["time"]
            duration = (end_time - start_time).total_seconds()
            sleep_seconds += duration

            # Salvare la sessione di sonno per il debug
            sleep_sessions.append({
                "night": night,
                "start_time": start_time,
                "end_time": end_time,
                "duration_seconds": duration
            })


        hours = int(sleep_seconds // 3600)
        minutes = int((sleep_seconds % 3600) // 60)
        seconds = int(sleep_seconds % 60)

        sleep_data.append({"night": night, "hours": hours, "minutes": minutes, "seconds": seconds})


    sleep_summary_df = pd.DataFrame(sleep_data)

    return sleep_summary_df

df_nights = sleep_per_night(df)


## CALCOLO MEDIE
def calculate_average_sleep(df):
    total_avg_seconds = df["hours"].mean() * 3600 + \
                        df["minutes"].mean() * 60 + \
                        df["seconds"].mean()

    average_hours = int(total_avg_seconds // 3600)
    average_minutes = int((total_avg_seconds % 3600) // 60)
    average_seconds = int(total_avg_seconds % 60)

    average_row = pd.DataFrame([{
        "night": "Media Totale",
        "hours": average_hours,
        "minutes": average_minutes,
        "seconds": average_seconds
    }])

    print(f"Media Totale: {average_hours} ore, {average_minutes} minuti e {average_seconds} secondi")
    return df

average = calculate_average_sleep(df_nights)
average

In [ ]:
def plot_corrected_sleep_data(df, start_time=None, end_time=None):
    df["time"] = pd.to_datetime(df["time"])

    # Filtrare il range temporale se specificato
    if start_time:
        start_time = pd.to_datetime(start_time).tz_localize("UTC", ambiguous='NaT', nonexistent='shift_forward')
        df = df[df["time"] >= start_time]

    if end_time:
        end_time = pd.to_datetime(end_time).tz_localize("UTC", ambiguous='NaT', nonexistent='shift_forward')
        df = df[df["time"] <= end_time]

    # Creare una colonna per visualizzare quando uno dorme
    df["sleeping"] = df["value"].apply(lambda x: 1 if x == 4095 else 0)


    plt.figure(figsize=(12, 5))
    plt.plot(df["time"], df["sleeping"], label="Stato del sonno (1=Dormendo, 0=Sveglio)", drawstyle="steps-post")
    plt.xlabel("Tempo")
    plt.ylabel("Stato del sonno")
    plt.title("Visualizzazione grafica del sonno")
    plt.yticks([0, 1], ["Sveglio", "Dormendo"])
    plt.legend()
    plt.grid()
    plt.show()



start_date = pd.Timestamp("2025-01-20")
end_date = pd.Timestamp("2025-01-22")
plot_corrected_sleep_data(df,start_time=start_date, end_time=end_date)

